In [1]:
import os

In [3]:

os.chdir("../")

In [54]:
%pwd

'e:\\Projects\\E2E Emotion Detection from text\\Emotion-Detection-using-ML'

In [56]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path_X: Path
    test_data_path_target: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path

In [57]:
from src.ML_emotion_detection.constants import *
from src.ML_emotion_detection.utils.common import read_yaml, create_directories, save_json

In [58]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.LogisticRegression
        # schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path_X=config.test_data_path_X,
            test_data_path_target=config.test_data_path_target,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            # target_column = schema.name
           
        )

        return model_evaluation_config

In [59]:
from sklearn.metrics import (accuracy_score, precision_score,
                             recall_score, f1_score)
from src.ML_emotion_detection.utils.common import load_bin

In [51]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        weighted_precision = precision_score(actual, pred, average= 'weighted')
        weighted_recall = recall_score(actual, pred, average= 'weighted')
        weighted_f1 = f1_score(actual, pred, average= 'weighted')
        return accuracy, weighted_precision, weighted_recall, weighted_f1
    
    def save_results(self):
        test_data_X = load_bin(self.config.test_data_path_X)
        test_data_target = load_bin(self.config.test_data_path_target)
        model = load_bin(self.config.model_path)
        
        
        predicted_output = model.predict(test_data_X)
        (acc, weighted_pre, weighted_rec, weighted_f1) = self.eval_metrics(test_data_target,
                                                                          predicted_output)

        scores = {"Accuracy": acc, "Weighted_precision": weighted_pre,
                  "Weighted_recall": weighted_rec, "Weighted_F1": weighted_f1}
        
        save_json(path=Path(self.config.metric_file_name), data=scores)        

In [53]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2024-11-24 22:58:26,393: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-24 22:58:26,401: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-24 22:58:26,407: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-11-24 22:58:26,410: INFO: common: created directory at: artifacts]
[2024-11-24 22:58:26,413: INFO: common: created directory at: artifacts/model_evaluation]
[2024-11-24 22:58:26,454: INFO: common: binary file loaded from: artifacts/data_transformation/X_test.joblib]
[2024-11-24 22:58:26,471: INFO: common: binary file loaded from: artifacts/data_transformation/y_test.joblib]
[2024-11-24 22:58:26,477: INFO: common: binary file loaded from: artifacts/model_trainer/log_reg_model.joblib]
[2024-11-24 22:58:26,655: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
